<a href="https://colab.research.google.com/github/secutron/FixMatch-pytorch/blob/master/FixMatch_pytorch_on_TPUs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FixMatch with Pytorch on TPUs - experimental

Code: https://github.com/vfdev-5/FixMatch-pytorch/


### Install requirements:

In [1]:
'''
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'


try:
  import torch_xla
  import ignite
except ImportError:
  VERSION = "20210304"    
  # VERSION = "nightly"
  # VERSION = "20200607"
  !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
  !python pytorch-xla-env-setup.py --version $VERSION
  !pip install --upgrade git+https://github.com/pytorch/ignite
  !pip install --upgrade --pre hydra-core
  !pip install wandb
  !pip uninstall -y pillow && CC="cc -mavx2" pip install --no-cache-dir --force-reinstall pillow-simd
'''

!pip install pytorch-ignite

!pip install hydra-core

     |████████████████████████████████| 232 kB 5.2 MB/s 
     |████████████████████████████████| 144 kB 5.0 MB/s 
     |████████████████████████████████| 112 kB 19.1 MB/s 
     |████████████████████████████████| 74 kB 2.6 MB/s 
     |████████████████████████████████| 636 kB 19.4 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141229 sha256=890140964329c59bad0e9286314c95fb7f7a65cc2c7ba67dfe41017603ffd3f4
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### Download dataset:

In [2]:
%cd /content/

/content


In [3]:
# Download dataset:
from torchvision.datasets import CIFAR10

CIFAR10("/content/cifar10", train=True, download=True);


Extracting /content/cifar10/cifar-10-python.tar.gz to /content/cifar10


### Get the code:

In [4]:
!git clone https://github.com/secutron/FixMatch-pytorch/

Cloning into 'FixMatch-pytorch'...
remote: Enumerating objects: 294, done.
remote: Counting objects: 100% (294/294), done.
remote: Compressing objects: 100% (193/193), done.
remote: Total 294 (delta 167), reused 164 (delta 76), pack-reused 0
Receiving objects: 100% (294/294), 112.20 KiB | 4.88 MiB/s, done.
Resolving deltas: 100% (167/167), done.


#### Optionally, login to `W&B`

To skip logging to `W&B`, please set `online_exp_tracking.wandb=false` 

In [5]:
# !wandb login <token>

### Let's train ResNet18 model in a faster mode

In [6]:
%pwd

'/content'

In [7]:

'''
!cd FixMatch-pytorch && export PYTHONPATH=$PWD:$PYTHONPATH && \
  python main_fixmatch.py distributed.backend=xla-tpu distributed.nproc_per_node=8 online_exp_tracking.wandb=true solver.num_epochs=1 \
    ssl.confidence_threshold=0.7 ema_decay=0.9 ssl.cta_update_every=15 solver.optimizer.params.lr=0.1
'''

%cd FixMatch-pytorch/

/content/FixMatch-pytorch


In [8]:
%pwd

'/content/FixMatch-pytorch'

In [9]:
!python main_fixmatch.py


/usr/local/lib/python3.7/dist-packages/hydra/core/default_element.py:127: UserWarning: In 'ssl/cta_pseudo': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  See {url} for more information"""
/usr/local/lib/python3.7/dist-packages/hydra/core/default_element.py:127: UserWarning: In 'solver/default': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  See {url} for more information"""
/usr/local/lib/python3.7/dist-packages/hydra/core/default_element.py:127: UserWarning: In 'dataflow/cifar10': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  See {url} for more information"""
2021-08-06 08:03:53,376 ignite.distributed.launcher.Parallel INFO: I

### Let's train WRN-28-2 model

In [ ]:
!cd FixMatch-pytorch && export PYTHONPATH=$PWD:$PYTHONPATH && \
  python -u main_fixmatch.py model=WRN-28-2 distributed.backend=xla-tpu distributed.nproc_per_node=8 online_exp_tracking.wandb=true